<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/aware/cifar10_aware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 483.4 MB 24 kB/s 
     |████████████████████████████████| 5.8 MB 43.7 MB/s 
     |████████████████████████████████| 2.3 MB 35.8 MB/s 
     |████████████████████████████████| 12.8 MB 145 kB/s 
     |████████████████████████████████| 1.3 MB 33.2 MB/s 
     |████████████████████████████████| 463 kB 71.2 MB/s 
     |████████████████████████████████| 211 kB 31.7 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

from tensorflow import keras
from keras.datasets import cifar10

In [ ]:
# load train and test dataset
def load_dataset():
# load dataset
  
  (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
  
  
	
  return train_images, train_labels, test_images, test_labels

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/cifar10.h5')

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
#Define Quantization Model

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLay  (None, 32, 32, 3)         3         
er)                                                              
_________________________________________________________________
quant_conv2d (QuantizeWrapp  (None, 32, 32, 64)        1923      
er)                                                              
_________________________________________________________________
quant_max_pooling2d (Quanti  (None, 16, 16, 64)        1         
zeWrapper)                                                       
_________________________________________________________________
quant_conv2d_1 (QuantizeWra  (None, 16, 16, 128)       74115     
pper)                                                            
_________________________________________________________________
quant_max_pooling2d_1 (Quan  (None, 8, 8, 128)         1

In [ ]:
#train and evaluate model against baseline
train_images_subset = train_images[0:1000]  # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=5, validation_split=0.1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1048: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 8s 3s/step - loss: 0.2199 - accuracy: 0.9256 - val_loss: 0.1433 - val_accuracy: 0.9500
Epoch 2/5
2/2 [==============================] - 6s 3s/step - loss: 0.0981 - accuracy: 0.9689 - val_loss: 0.1045 - val_accuracy: 0.9400
Epoch 3/5
2/2 [==============================] - 6s 3s/step - loss: 0.0522 - accuracy: 0.9811 - val_loss: 0.0783 - val_accuracy: 0.9500
Epoch 4/5
2/2 [==============================] - 6s 3s/step - loss: 0.0337 - accuracy: 0.9878 - val_loss: 0.0580 - val_accuracy: 0.9900
Epoch 5/5
2/2 [==============================] - 6s 3s/step - loss: 0.0224 - accuracy: 0.9911 - val_loss: 0.0389 - val_accuracy: 1.0000


In [ ]:
#Create Quantize model for TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2fzxd_cd/assets


INFO:tensorflow:Assets written to: /tmp/tmp2fzxd_cd/assets


In [ ]:
#Evaluate the quantized model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1048: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Baseline test accuracy: 0.7448999881744385
Quant test accuracy: 0.7279000282287598


In [ ]:
#Check the size of the models
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: /tmp/tmp1ilk2hqd/assets


INFO:tensorflow:Assets written to: /tmp/tmp1ilk2hqd/assets


Float model in Mb: 19.443763732910156
Quantized model in Mb: 4.8814239501953125
